In [1]:
import gensim
import pandas as pd
import re
from tqdm import tqdm
import numpy as np
import nltk
from multiprocessing import cpu_count

In [2]:
from string import punctuation

In [3]:
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»—«–'

In [4]:
from nltk.corpus import stopwords 
stops = stopwords.words('russian') + list(punctuation) + ['»', '—', '«', '–']

In [5]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [6]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [7]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'

In [8]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))

In [9]:
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)

In [10]:
russian.shape

(595931, 8)

In [11]:
russian.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь


In [12]:
teams1_match = np.zeros(russian.shape[0])
for i, team1 in enumerate(russian['team_of_season_1_name_in_russian']):
    if team1 in russian['result.tsv'].values[i] :
        teams1_match[i] = 1 
    else: 
        teams1_match[i] = 0
russian['direct_match_for_teams1'] = teams1_match

In [13]:
teams2_match = np.zeros(russian.shape[0])
for i, team2 in enumerate(russian['team_of_season_2_name_in_russian']):
    if team2 in russian['result.tsv'].values[i] :
        teams2_match[i] = 1 
    else: 
        teams2_match[i] = 0
russian['direct_match_for_teams2'] = teams2_match

In [14]:
leagues_match = np.zeros(russian.shape[0])
for i, leagues in enumerate(russian['league_name_in_russian']):
    if leagues in russian['result.tsv'].values[i] :
        leagues_match[i] = 1 
    else: 
        leagues_match[i] = 0
russian['direct_match_for_leagues'] = leagues_match

In [15]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [word for word in words if word and word not in stops]

    return words

In [16]:
%%time 
russian['content_norm'] = russian['result.tsv'].apply(normalize)

Wall time: 6min 24s


In [17]:
russian.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,direct_match_for_teams1,direct_match_for_teams2,direct_match_for_leagues,content_norm
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан,1.0,1.0,0.0,"[сборная, турции, продолжает, дорогу, чемпиона..."
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили,1.0,1.0,0.0,"[двадцать, девятого, числа, состоится, весьма,..."
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь,0.0,1.0,0.0,"[аз, алкмар, первом, матче, голландский, клуб,..."


In [20]:
w2v_model = gensim.models.Word2Vec([text for text in tqdm(russian['content_norm'])], size = 300, workers=cpu_count())

100%|██████████████████████████████████████████████████████████████████████| 595931/595931 [00:00<00:00, 627567.29it/s]


In [28]:
w2v_model.save('w2v_sport_model.model')

In [21]:
fasttxt_model = gensim.models.FastText([text for text in tqdm(russian['content_norm'])], size = 300, workers=cpu_count())

100%|██████████████████████████████████████████████████████████████████████| 595931/595931 [00:00<00:00, 792364.37it/s]


In [30]:
fasttxt_model.save('fasttext_sport_model.model')

In [22]:
from nltk import word_tokenize

In [28]:
for i, text in enumerate(russian['content_norm'][10:20].values):
  # print(i, text)
    for word in text:
    # print(word)
        word2 = russian['team_of_season_2_name_in_russian'][i]
        word2 = word2.lower()
        if model_fast.similarity(word, word2) > 0.4:
            print(word, word2)
            print(model_fast.similarity(word, word2))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


филипс узбекистан


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  if __name__ == '__main__':


0.44604123
фейеноорд айнтрахт франкфурт
0.43834028
зволле айнтрахт франкфурт
0.43804717
фейеноорд айнтрахт франкфурт
0.43834028
цска краснодар
0.6780505
байера краснодар
0.40357396
армейцы краснодар
0.46718252
лудогорца краснодар
0.4809531
цска краснодар
0.6780505
болгарский краснодар
0.4002518
дортмунд бавария
0.690273
германия бавария
0.62242275
боруссия бавария
0.74700695
барселону бавария
0.48215353
шмелей бавария
0.4445119
барселоны бавария
0.46646658
боруссия бавария
0.74700695
бундес-лиге бавария
0.45489377
немецкого бавария
0.4046752
лейпцига бавария
0.60683316
бенфика бавария
0.5863577
порту бавария
0.46518242
бенфика бавария
0.5863577
апл челси
0.525126
ливерпуль челси
0.67570496
апл челси
0.525126
норвич челси
0.6380881
ливерпуль челси
0.67570496
апл челси
0.525126
норвич челси
0.6380881
энфилде?я челси
0.55790734
ливер челси
0.636581
англичане эстония
0.43274766


In [38]:
print(model_w2v.most_similar('апл'))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('бернли', 0.6764350533485413), ('брайтон', 0.6657365560531616), ('бёрнли', 0.6609916090965271), ('чемпионшипа', 0.6573724150657654), ('уотфорд', 0.6481508612632751), ('саутгемптон', 0.644432783126831), ('фулхэм', 0.6414387226104736), ('чемпионшип', 0.632818341255188), ('лестер', 0.631027340888977), ('чемпионшипе', 0.6297529935836792)]


# FastText

### Команда 1 

In [18]:
model2 = 'fasttext_sport_model.model'
model_fast = gensim.models.FastText.load(model2)

In [19]:
teams1_vector_match = np.zeros(russian.shape[0])

In [23]:
for i, text in enumerate(russian['content_norm'].values):
    for word in text:
        try:
            word2 = russian['team_of_season_1_name_in_russian'][i]
            word2 = word2.lower()
            #print(word2)
            if model_fast.similarity(word, word2) > 0.7:
                teams1_vector_match[i] = 1
        except:
            continue

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  import sys


In [24]:
russian['ft_vector_match_team1'] = teams1_vector_match

In [25]:
all_vals = russian['ft_vector_match_team1'].shape[0]
print(all_vals)

595931


In [26]:
success = russian[russian['ft_vector_match_team1'] == 1].shape[0]
print(success)

18644


In [27]:
vectorline_team1 = success / all_vals
print(vectorline_team1)

0.03128550117379361


### Команда 2 

In [28]:
teams2_vector_match = np.zeros(russian.shape[0])

In [29]:
for i, text in enumerate(russian['content_norm'].values):
    for word in text:
        try:
            word2 = russian['team_of_season_2_name_in_russian'][i]
            word2 = word2.lower()
            if model_fast.similarity(word, word2) > 0.7:
                teams2_vector_match[i] = 1
        except:
            continue

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [30]:
russian['ft_vector_match_team2'] = teams2_vector_match

In [31]:
all_vals2 = russian['ft_vector_match_team2'].shape[0]
print(all_vals2)

595931


In [32]:
success2 = russian[russian['ft_vector_match_team2'] == 1].shape[0]
print(success2)

18416


In [33]:
vectorline_team2 = success2 / all_vals2
print(vectorline_team2)

0.03090290654454962


### Лига

In [34]:
league_vector_match = np.zeros(russian.shape[0])

In [35]:
for i, text in enumerate(russian['content_norm'].values):
    for word in text:
        try:
            word2 = russian['league_name_in_russian'][i]
            word2 = word2.lower()
            if model_fast.similarity(word, word2) > 0.7:
                league_vector_match[i] = 1
        except:
            continue

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [36]:
russian['ft_vector_match_league'] = league_vector_match

In [37]:
all_vals3 = russian['ft_vector_match_league'].shape[0]
print(all_vals3)

595931


In [38]:
success3 = russian[russian['ft_vector_match_league'] == 1].shape[0]
print(success3)

18074


In [39]:
vectorline_league = success3 / all_vals3
print(vectorline_league)

0.030329014600683636


In [40]:
russian.to_csv(r'C:\Users\User\Desktop\project\russian_vectormodels.csv', index = False)

# W2V

In [56]:
model1 = 'w2v_sport_model.model'
model_w2v = gensim.models.Word2Vec.load(model1)

In [57]:
teams1_vector_match_w2v = np.zeros(russian.shape[0])

In [62]:
for i, text in enumerate(russian['content_norm'][:50].values):
    for word in text:
        word2 = russian['team_of_season_1_name_in_russian'][i]
        word2 = word2.lower()
        try:
            if model_w2v.similarity(word, word2) > 0.5:
                print(word, word2)
                print(model_w2v.similarity(word, word2))
        except:
            continue

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


сборная турция


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


0.5222218
турции турция
0.6121782
узбекистан турция
0.50324047
турки турция
0.56477135
турция турция
0.99999994
францией турция
0.59041035
исландию турция
0.6758712
францией турция
0.59041035
турки турция
0.56477135
узбекистан турция
0.50324047
сборная турция
0.5222218
сборная турция
0.5222218
сборная турция
0.5222218
турции турция
0.6121782
узбекистан турция
0.50324047
турки турция
0.56477135
америки колумбия
0.5210236
колумбия колумбия
1.0
чили колумбия
0.7665932
кофейщики колумбия
0.5159029
парагвая колумбия
0.5880945
аргентина колумбия
0.862471
кофейщики колумбия
0.5159029
уругвая колумбия
0.6390867
чилийцам колумбия
0.50081825
абердин флора
0.5144291
флора флора
1.0
флора флора
1.0
хеккен флора
0.51407427
хеккен флора
0.51407427
валенсия валенсия
1.0
валенсия валенсия
1.0
валенсия валенсия
1.0
ахмат ахмат
1.0
краснодар ахмат
0.7594689
рпл ахмат
0.5679152
ахмат ахмат
1.0
цска ахмат
0.7137059
ахмат ахмат
1.0
краснодар ахмат
0.7594689
рубина ахмат
0.6412567
краснодар ахмат
0.7594689


KeyError: 18

In [54]:
for i, text in enumerate(russian['content_norm'][:20].values):
    for word in text:
        try:
            word2 = russian['team_of_season_1_name_in_russian'][i]
            word2 = word2.lower()
            #print(word2)
            if model_w2v.similarity(word, word2) > 0.4:
                teams1_vector_match_w2v[i] = 1
                print(word, word2)
        except:
            continue

In [47]:
russian['w2v_vector_match_team1'] = teams1_vector_match_w2v

In [48]:
all_vals4 = russian['w2v_vector_match_team1'].shape[0]
print(all_vals4)
success4 = russian[russian['w2v_vector_match_team1'] == 1].shape[0]
print(success4)
vectorline_league = success4 / all_vals4
print(vectorline_league)

595931
0
0.0


In [43]:
teams2_vector_match_w2v = np.zeros(russian.shape[0])

In [44]:
for i, text in enumerate(russian['content_norm'].values):
    for word in text:
        try:
            word2 = russian['team_of_season_2_name_in_russian'][i]
            word2 = word2.lower()
            #print(word2)
            if model_w2v.similarity(word, word2) > 0.7:
                teams2_vector_match_w2v[i] = 1
        except:
            continue

In [49]:
russian['w2v_vector_match_team2'] = teams2_vector_match_w2v

In [50]:
all_vals5 = russian['w2v_vector_match_team2'].shape[0]
print(all_vals5)
success5 = russian[russian['w2v_vector_match_team2'] == 1].shape[0]
print(success5)
vectorline_league = success5 / all_vals5
print(vectorline_league)

595931
0
0.0


In [45]:
league_vector_match_w2v = np.zeros(russian.shape[0])

In [46]:
for i, text in enumerate(russian['content_norm'].values):
    for word in text:
        try:
            word2 = russian['league_name_in_russian'][i]
            word2 = word2.lower()
            if model_w2v.similarity(word, word2) > 0.7:
                league_vector_match_w2v[i] = 1
        except:
            continue

In [51]:
russian['w2v_vector_match_league'] = league_vector_match_w2v

In [52]:
all_vals6 = russian['w2v_vector_match_league'].shape[0]
print(all_vals6)
success6 = russian[russian['w2v_vector_match_league'] == 1].shape[0]
print(success6)
vectorline_league = success6 / all_vals6
print(vectorline_league)

595931
0
0.0
